from here

In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [4]:
base_path = "/kaggle/input/groundnut-plant-leaf-data/Dataset of groundnut plant leaf images for classification and detection/Groundnut_Leaf_dataset"
train_dir = os.path.join(base_path, "train")
test_dir = os.path.join(base_path, "test")

img_size = 224
batch_size = 32
num_epochs = 10
learning_rate = 1e-4


In [5]:
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], 
                         [0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(train_dir, transform=transform)
test_dataset = datasets.ImageFolder(test_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class_names = train_dataset.classes
print(f"Classes: {class_names}")


Classes: ['early_leaf_spot_1', 'early_rust_1', 'healthy_leaf_1', 'late_leaf_spot_1', 'nutrition_deficiency_1', 'rust_1']


In [6]:
label_encoder = LabelEncoder()
label_encoder.fit(class_names)

# Save encoder
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)


In [7]:
model = models.mobilenet_v2(pretrained=True)
for param in model.parameters():
    param.requires_grad = False  # Freeze base

# Replace classifier
num_classes = len(class_names)
model.classifier[1] = nn.Linear(model.last_channel, num_classes)
model = model.to(device)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 121MB/s]


In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier[1].parameters(), lr=learning_rate)


In [9]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    accuracy = correct / total
    print(f"Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}")


Epoch 1/10: 100%|██████████| 248/248 [01:42<00:00,  2.42it/s]


Loss: 1.2565, Accuracy: 0.6171


Epoch 2/10: 100%|██████████| 248/248 [00:39<00:00,  6.35it/s]


Loss: 0.7362, Accuracy: 0.8326


Epoch 3/10: 100%|██████████| 248/248 [00:38<00:00,  6.45it/s]


...

Loss: 0.3501, Accuracy: 0.8946


Epoch 9/10: 100%|██████████| 248/248 [00:37<00:00,  6.69it/s]


Loss: 0.3358, Accuracy: 0.8984


Epoch 10/10: 100%|██████████| 248/248 [00:37<00:00,  6.60it/s]

Loss: 0.3179, Accuracy: 0.9013


In [10]:
torch.save(model.state_dict(), "groundnut_mobilenet.pth")
print("Model saved to groundnut_mobilenet.pth")


Model saved to groundnut_mobilenet.pth


In [11]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

print(f"Test Accuracy: {correct / total:.4f}")


Test Accuracy: 0.9649


In [12]:
from PIL import Image

# === Load Unseen Image(s) ===
def predict_unseen_image(image_path, model, transform, class_names):
    model.eval()
    
    image = Image.open(image_path).convert("RGB")
    input_tensor = transform(image).unsqueeze(0).to(device)  # Add batch dimension
    
    with torch.no_grad():
        outputs = model(input_tensor)
        probabilities = torch.nn.functional.softmax(outputs[0], dim=0)
        predicted_index = torch.argmax(probabilities).item()
        predicted_label = class_names[predicted_index]
        confidence = probabilities[predicted_index].item()
    
    return predicted_label, confidence


In [14]:
# Load label encoder (if you saved it)
with open("label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

class_names = label_encoder.classes_

# Define transform (same as training)
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], 
                         [0.229, 0.224, 0.225])
])

# Predict a single image
unseen_image_path = "/kaggle/input/groundnut-plant-leaf-data/Dataset of groundnut plant leaf images for classification and detection/Groundnut_Leaf_dataset/test/healthy_leaf_1/42.jpg"  # change this
predicted_class, confidence = predict_unseen_image(unseen_image_path, model, transform, class_names)

print(f"Prediction: {predicted_class} ({confidence*100:.2f}% confidence)")


Prediction: healthy_leaf_1 (93.46% confidence)


In [15]:
# Load label encoder (if you saved it)
with open("label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

class_names = label_encoder.classes_

# Define transform (same as training)
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], 
                         [0.229, 0.224, 0.225])
])

# Predict a single image
unseen_image_path = "/kaggle/input/groundnut-plant-leaf-data/Dataset of groundnut plant leaf images for classification and detection/Groundnut_Leaf_dataset/test/early_leaf_spot_1/IMG_2135.jpg"  # change this
predicted_class, confidence = predict_unseen_image(unseen_image_path, model, transform, class_names)

print(f"Prediction: {predicted_class} ({confidence*100:.2f}% confidence)")


Prediction: early_leaf_spot_1 (46.83% confidence)


In [16]:
# Load label encoder (if you saved it)
with open("label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

class_names = label_encoder.classes_

# Define transform (same as training)
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], 
                         [0.229, 0.224, 0.225])
])

# Predict a single image
unseen_image_path = "/kaggle/input/groundnut-plant-leaf-data/Dataset of groundnut plant leaf images for classification and detection/Groundnut_Leaf_dataset/test/early_rust_1/IMG_8942.jpg"  # change this
predicted_class, confidence = predict_unseen_image(unseen_image_path, model, transform, class_names)

print(f"Prediction: {predicted_class} ({confidence*100:.2f}% confidence)")


Prediction: early_rust_1 (93.66% confidence)


In [17]:
# Load label encoder (if you saved it)
with open("label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

class_names = label_encoder.classes_

# Define transform (same as training)
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], 
                         [0.229, 0.224, 0.225])
])

# Predict a single image
unseen_image_path = "/kaggle/input/groundnut-plant-leaf-data/Dataset of groundnut plant leaf images for classification and detection/Groundnut_Leaf_dataset/test/late_leaf_spot_1/dr_0_3979.jpg"  # change this
predicted_class, confidence = predict_unseen_image(unseen_image_path, model, transform, class_names)

print(f"Prediction: {predicted_class} ({confidence*100:.2f}% confidence)")


Prediction: late_leaf_spot_1 (96.91% confidence)


In [18]:
# Load label encoder (if you saved it)
with open("label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

class_names = label_encoder.classes_

# Define transform (same as training)
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], 
                         [0.229, 0.224, 0.225])
])

# Predict a single image
unseen_image_path = "/kaggle/input/groundnut-plant-leaf-data/Dataset of groundnut plant leaf images for classification and detection/Groundnut_Leaf_dataset/test/nutrition_deficiency_1/dr_0_2432.jpg"  # change this
predicted_class, confidence = predict_unseen_image(unseen_image_path, model, transform, class_names)

print(f"Prediction: {predicted_class} ({confidence*100:.2f}% confidence)")


Prediction: nutrition_deficiency_1 (86.92% confidence)


In [19]:
# Load label encoder (if you saved it)
with open("label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

class_names = label_encoder.classes_

# Define transform (same as training)
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], 
                         [0.229, 0.224, 0.225])
])

# Predict a single image
unseen_image_path = "/kaggle/input/groundnut-plant-leaf-data/Dataset of groundnut plant leaf images for classification and detection/Groundnut_Leaf_dataset/test/rust_1/IMG_9015.jpg"  # change this
predicted_class, confidence = predict_unseen_image(unseen_image_path, model, transform, class_names)

print(f"Prediction: {predicted_class} ({confidence*100:.2f}% confidence)")


Prediction: rust_1 (99.91% confidence)


In [20]:
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)
print("✅ Label encoder saved as label_encoder.pkl")

✅ Label encoder saved as label_encoder.pkl
